# analysis.final.b.step2

This script creates important tables and figures

In [1]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

incidence_vars = [
    'Total Mortality(low estimate)',
    'Total Mortality(high estimate)',
    'PM Mortality, All Cause (low)',
    'PM Mortality, All Cause (high)',
    'PM Infant Mortality',
    'Total O3 Mortality',
    'O3 Mortality (Short-term exposure)',
    'O3 Mortality (Long-term exposure)',
    'Total Asthma Symptoms',
    'PM Asthma Symptoms, Albuterol use',
    'O3 Asthma Symptoms, Chest Tightness',
    'O3 Asthma Symptoms, Cough',
    'O3 Asthma Symptoms, Shortness of Breath',
    'O3 Asthma Symptoms, Wheeze',
    'Total Incidence, Asthma',
    'PM Incidence, Asthma',
    'O3 Incidence, Asthma',
    'Total Incidence, Hay Fever/Rhinitis',
    'PM Incidence, Hay Fever/Rhinitis',
    'O3 Incidence, Hay Fever/Rhinitis',
    'Total ER Visits, Respiratory',
    'PM ER Visits, Respiratory',
    'O3 ER Visits, Respiratory',
    'Total Hospital Admits, All Respiratory',
    'PM Hospital Admits, All Respiratory',
    'O3 Hospital Admits, All Respiratory',
    'PM Nonfatal Heart Attacks',
    'PM Minor Restricted Activity Days',
    'PM Work Loss Days',
    'PM Incidence Lung Cancer',
    'PM HA Cardio Cerebro and Peripheral Vascular Disease',
    'PM HA Alzheimers Disease',
    'PM HA Parkinsons Disease',
    'PM Incidence Stroke',
    'PM Incidence Out of Hospital Cardiac Arrest',
    'PM ER visits All Cardiac Outcomes',
    'O3 ER Visits, Asthma',
    'O3 School Loss Days, All Cause'
]


In [2]:
from google.colab import drive
import os

drive.mount('/content/drive') # Comment this out if running ipynb locally
wdir = '/content/drive/MyDrive/gpDept-ResearchDept/LNG Air Pollution/LNG Health - COBRA project/git_repo/Permit-To-Kill-COBRA-Research/' # Replace this with your working directory path
os.chdir(wdir)

Mounted at /content/drive


In [3]:
"""
Read in combined results
"""

# Script ======================================================================
results_dir0 = "Version 5 analysis"
agg_df_f0 = results_dir0 + "/b.finalData.results/b.finalData.01-03.combined_results.csv"
agg_df3 = pd.read_csv(agg_df_f0)
agg_df3['Project Status'] = pd.Categorical(agg_df3['Project Status'], categories=['Operating', 'Under Construction', 'Planned'])

In [4]:
pip install kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.9 MB/s eta 0:00:00


## Project-level analysis

**Briefing Table 3.** Estimated single-year health impacts by LNG project

In [5]:
"""
Single-Year Health Impacts
Table export
-----------------
Sorted by project status first
Then sorted by project-level mortality or terminal-level mortality, split by project
"""

# Table configuration ========================================================
tbl_yr = 2023
f_out = 'b.finalData.briefing_table.project_level_results.xlsx'
save_to_xlsx = True
million_usd_unit = True
sort_by = 'Project mortality' # Options: Terminal mortality, Project mortality, Alphabetical, Alphabetical/Status

pivot_indices = ['Project', 'Project Status', 'Terminal']
colset0 = ['Total Mortality(high estimate)', 'Total Mortality(low estimate)',
        '$ Total Health Benefits(high estimate)', '$ Total Health Benefits(low estimate)',
        'Total Incidence, Asthma',
          #  '$ Total Incidence, Asthma',
        'Total Asthma Symptoms',
          #  '$ Total Asthma Symptoms',
        'PM Work Loss Days', 'O3 School Loss Days, All Cause'
]

# colset0 = [i for i in agg_df3.columns if (('Total' in i) and '$' not in i)]
# colset0 = [i for i in agg_df3.columns if ('Total' in i)]

# Script =====================================================================

colsetf = pivot_indices + colset0

# Table scripts ==============================================================
# Filter agg_df3 down to just the rows and fields we need
tbl1_0 = agg_df3[(agg_df3['Analysis Year'] == tbl_yr)][colsetf]

# Create metadata DF for merging pivoted tables
add_meta = tbl1_0[pivot_indices].drop_duplicates()

# Sum by project and convert negatives to positive
tbl1_1 = -pd.pivot_table(tbl1_0[(['Project'] + colset0)],
               index=['Project'],
               aggfunc="sum")

if sort_by == 'Terminal mortality':
    # Create another table that is summed by terminal
    tbl1_2 = -pd.pivot_table(tbl1_0[(['Terminal'] + colset0)],
                  index=['Terminal'],
                            values = ['Total Mortality(high estimate)', '$ Total Health Benefits(high estimate)'],
                            aggfunc="sum")

    # Rename columns for legibility
    tbl1_2.rename(
        columns = {'Total Mortality(high estimate)': 'Terminal Total Mortality(high estimate)',
                  '$ Total Health Benefits(high estimate)': '$ Terminal Total Health Benefits(high estimate)'
                  }, inplace=True)

    # Merge the project-level data with the terminal-level sums for table sorting
    tbl1_3 = (tbl1_1
              .merge(add_meta, left_on=tbl1_1.index, right_on='Project')
              .merge(tbl1_2, left_on='Terminal', right_on=tbl1_2.index)
    )

    # Sort by project status first and then terminal total mortality (so that projects of the same terminal are grouped together)
    tbl1_3.sort_values(by=['Project Status', 'Terminal Total Mortality(high estimate)'], ascending=[True, False], inplace=True)

elif sort_by == 'Project mortality':
    tbl1_3 = (tbl1_1
              .merge(add_meta, left_on=tbl1_1.index, right_on='Project')
    )
    tbl1_3.sort_values(by=['Project Status', 'Total Mortality(high estimate)'], ascending=[True, False], inplace=True)

elif sort_by == 'Alphabetical':
    tbl1_3 = (tbl1_1
              .merge(add_meta, left_on=tbl1_1.index, right_on='Project')
    )
    tbl1_3.sort_values(by=['Project'], inplace=True)

elif sort_by == 'Alphabetical/Status':
    tbl1_3 = (tbl1_1
              .merge(add_meta, left_on=tbl1_1.index, right_on='Project')
    )
    tbl1_3.sort_values(by=['Project Status', 'Project'], inplace=True)

tbl1_4 = tbl1_3[(pivot_indices + colset0)]
tbl1_5 = tbl1_4.drop(columns=['Terminal'])

if million_usd_unit:
    usd_unit = [i for i in colset0 if ('$'  in i)]
    for i in usd_unit:
        tbl1_5[i] = tbl1_5[i] / 1000000
        tbl1_5.rename(columns={i: f'{i} (million USD)'}, inplace=True)

if save_to_xlsx:
    xlsx_out = results_dir0 + "/b.finalData.results/" + f_out
    tbl1_5.to_excel(xlsx_out, index=False)

tbl1_5.head(5)

,Project,Project Status,Total Mortality(high estimate),Total Mortality(low estimate),$ Total Health Benefits(high estimate) (million USD),$ Total Health Benefits(low estimate) (million USD),"Total Incidence, Asthma",Total Asthma Symptoms,PM Work Loss Days,"O3 School Loss Days, All Cause"
28,Sabine Pass LNG Phase I,Operating,15.978999,13.006942,257.029691,213.649877,81.952719,12620.063905,319.109334,7032.201060
29,Sabine Pass LNG Phase II,Operating,7.988900,6.502891,128.504474,106.814841,40.965768,6309.651328,159.554130,3515.722554
3,Cameron LNG Phase I,Operating,7.895032,5.593856,124.158864,90.571142,31.970255,5005.983746,245.433006,2495.456260
10,Cove Point LNG,Operating,7.771505,4.322651,117.901895,67.562791,18.081327,3023.066936,418.535316,962.822720
7,Corpus Christi LNG Stage I,Operating,7.738554,6.446611,128.005790,109.148722,51.960262,7881.293909,162.341922,4485.247807


**Briefing Figure 1.** Single-year mortality estimates by LNG terminal

In [8]:
"""
Project-level Health Impacts Bar Chart (high)
"""

import plotly.express as px
import plotly.graph_objects as go

color_map = {
    status: color for status, color in zip(tbl1_4['Project Status'].unique(), ['#003B4A', '#D54400', '#F7BE00'])
}


tbl1_4.sort_values(by=['Project Status'], inplace=True)

fig2 = px.bar(tbl1_4, y='Terminal', x="Total Mortality(high estimate)", color='Project Status', color_discrete_map=color_map)
fig2.update_layout(yaxis={'categoryorder':'total ascending'},

                   height=600, width=1000,
                                     legend=dict(
                            yanchor="bottom",
                            y=0.12,
                            xanchor="right",
                            x=0.92
                        ),
                   xaxis=dict(title='Premature Deaths (high estimate)')
                   )

fig2.update_layout(font=dict(size=15))
fig2.write_image(results_dir0 + "/b.finalData.results/figure_2_project_impacts.svg", engine="kaleido")
fig2.write_image(results_dir0 + "/b.finalData.results/figure_2_project_impacts.pdf", engine="kaleido")
fig2.write_image(results_dir0 + "/b.finalData.results/figure_2_project_impacts.jpg")

fig2.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1971: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [10]:
"""
Appendix: Project-level Health Impacts Bar Chart (low)
"""

import plotly.express as px
import plotly.graph_objects as go

color_map = {
    status: color for status, color in zip(tbl1_4['Project Status'].unique(), ['#003B4A', '#D54400', '#F7BE00'])
}


tbl1_4.sort_values(by=['Project Status'], inplace=True)

fig2a = px.bar(tbl1_4, y='Terminal', x="Total Mortality(low estimate)", color='Project Status', color_discrete_map=color_map)
fig2a.update_layout(yaxis={'categoryorder':'total ascending'},

                   height=600, width=1000,
                                     legend=dict(
                            yanchor="bottom",
                            y=0.12,
                            xanchor="right",
                            x=0.92
                        ),
                                      xaxis=dict(title='Premature Deaths (low estimate)')

                   )

fig2a.write_image(results_dir0 + "/b.finalData.results/appendix_figure_a1_project_impacts-low.svg", engine="kaleido")
fig2a.write_image(results_dir0 + "/b.finalData.results/appendix_figure_a1_project_impacts-low.pdf", engine="kaleido")
fig2a.write_image(results_dir0 + "/b.finalData.results/appendix_figure_a1_project_impacts-low.jpg")

fig2a.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1971: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## County-level analysis

In [14]:
"""
Impacts by county (Operating Projects only)
Table export
---------------
Sorted by county with the greatest total health impacts
Then, within the county, subtotals by project status
"""

# Table configuration ========================================================
tbl_yr = 2023
save_to_xlsx = True
f_out1 = 'b.finalData.briefing_table.county_level_results-sorted_by_total.xlsx'
f_out2 = 'b.finalData.briefing_table.county_level_results-sorted_by_percapita.xlsx'
million_usd_unit = True

pivot_indices = ['Destination County', 'Destination State', 'Project Status', 'Terminal']

# colset0 = ['Total Mortality(high estimate)', 'Total Mortality(low estimate)',
#         'Total Mortality(high estimate) PER MILLION', 'Total Mortality(low estimate) PER MILLION',
#         'Delta PM 2.5', 'Delta O3',
#         '$ Total Health Benefits(high estimate)', '$ Total Health Benefits(low estimate)'
# ]
colset0 = ['Total Mortality(high estimate)', 'Total Mortality(low estimate)',
        'Total Mortality(high estimate) PER MILLION', 'Total Mortality(low estimate) PER MILLION',
        '$ Total Health Benefits(high estimate)', '$ Total Health Benefits(low estimate)',
        'Total Incidence, Asthma',
          #  '$ Total Incidence, Asthma',
        'Total Asthma Symptoms',
           # '$ Total Asthma Symptoms',
        'PM Work Loss Days', 'O3 School Loss Days, All Cause'
]

colsetf = pivot_indices + colset0

# Table script ===============================================================

tbl2_0 = agg_df3[((agg_df3['Analysis Year'] == tbl_yr) & (agg_df3['Project Status'] == 'Operating'))][colsetf]
tbl2_0['County, State'] = tbl2_0[['Destination County', 'Destination State']].agg(', '.join, axis=1)
tbl2_0.drop(['Destination County', 'Destination State'], axis=1, inplace=True)

tbl2_1 = -pd.pivot_table(tbl2_0[(['County, State'] + colset0)],
               index=['County, State'],
               aggfunc="sum")[colset0]

# Add column for most impacting LNG terminal
tbl2_2 = -pd.pivot_table(tbl2_0[(['County, State', 'Terminal'] + colset0)],
               index=['County, State', 'Terminal'],
               aggfunc="sum")[colset0]
tbl2_2.reset_index(inplace=True)
tbl2_2.sort_values(by=['$ Total Health Benefits(high estimate)'], ascending=False, inplace=True)

def get_most_impacting_terminals(group):
    n_terminals = 3
    group_total = group['$ Total Health Benefits(high estimate)'].sum()
    str_all = []

    for i in np.arange(n_terminals):
        terminal_name = group.iloc[i]['Terminal']
        terminal_per = group.iloc[i]['$ Total Health Benefits(high estimate)']/group_total * 100
        str_out = f'{terminal_name} ({terminal_per:.0f}%)'
        str_out = str_out.replace(' LNG', '')
        str_all.append(str_out)

    return (', '.join(str_all))

max_terminal = tbl2_2.groupby('County, State').apply(get_most_impacting_terminals)
max_terminal.name = 'Most impacting terminals'

tbl2_1 = tbl2_1.merge(max_terminal, left_on = tbl2_1.index, right_on=max_terminal.index)
tbl2_1.rename(columns = {'key_0': 'County, State'}, inplace=True)

by_total = tbl2_1.sort_values(by=['$ Total Health Benefits(high estimate)'], ascending=False)
by_intenstity = tbl2_1.sort_values(by=['Total Mortality(high estimate) PER MILLION'], ascending=False)

if million_usd_unit:
    usd_unit = [i for i in colset0 if ('$'  in i)]
    for i in usd_unit:
        by_total[i] = by_total[i] / 1000000
        by_intenstity[i] = by_intenstity[i] / 1000000
        by_total.rename(columns={i: f'{i} (million USD)'}, inplace=True)
        by_intenstity.rename(columns={i: f'{i} (million USD)'}, inplace=True)

if save_to_xlsx:
  xlsx_out1 = results_dir0 + "/b.finalData.results/" + f_out1
  xlsx_out2 = results_dir0 + "/b.finalData.results/" + f_out2
  by_total.to_excel(xlsx_out1, index=False)
  # by_intenstity.to_excel(xlsx_out2, index=False)

